# Case Study 1: Phân loại chữ số viết tay MNIST
## Tạ Cao Sơn - B22DCVT445

### Mục tiêu
- Phân loại chữ số viết tay (0-9) từ tập dữ liệu MNIST
- So sánh 2 cách triển khai CNN:
  - **Model 1**: CNN với Keras (high-level API)
  - **Model 2**: CNN từ đầu không dùng Keras (NumPy thuần)
- Tạo 100 ảnh test tự vẽ và đánh giá model

### Nội dung
1. Load và phân tích dữ liệu MNIST
2. Tạo 100 ảnh chữ số tự vẽ
3. Xây dựng CNN Model 1 (Keras)
4. Xây dựng CNN Model 2 (NumPy - không Keras)
5. So sánh và đánh giá 2 models

In [ ]:
# ======================================
# 1. Import Libraries & Setup
# ======================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Keras imports
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Set random seeds
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"TensorFlow/Keras version: {keras.__version__}")

## Phần 1: Load và Phân tích dữ liệu MNIST

In [ ]:
# ======================================
# 2. Load MNIST Dataset
# ======================================

print("📥 Loading MNIST dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"\n📊 Dataset Information:")
print(f"Training set: {X_train.shape} - Labels: {y_train.shape}")
print(f"Test set: {X_test.shape} - Labels: {y_test.shape}")
print(f"Image shape: {X_train[0].shape}")
print(f"Pixel value range: [{X_train.min()}, {X_train.max()}]")
print(f"Number of classes: {len(np.unique(y_train))}")
print(f"Classes: {np.unique(y_train)}")

In [ ]:
# ======================================
# 3. Phân bố dữ liệu (Data Distribution)
# ======================================

# Đếm số lượng mẫu của mỗi class
train_counts = np.bincount(y_train)
test_counts = np.bincount(y_test)

# Vẽ phân bố
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train distribution
axes[0].bar(range(10), train_counts, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Chữ số (Digit)', fontsize=12)
axes[0].set_ylabel('Số lượng mẫu', fontsize=12)
axes[0].set_title('Phân bố dữ liệu Training Set', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(10))
axes[0].grid(axis='y', alpha=0.3)
for i, count in enumerate(train_counts):
    axes[0].text(i, count + 100, str(count), ha='center', fontsize=10)

# Test distribution
axes[1].bar(range(10), test_counts, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('Chữ số (Digit)', fontsize=12)
axes[1].set_ylabel('Số lượng mẫu', fontsize=12)
axes[1].set_title('Phân bố dữ liệu Test Set', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(10))
axes[1].grid(axis='y', alpha=0.3)
for i, count in enumerate(test_counts):
    axes[1].text(i, count + 20, str(count), ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('mnist_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Phân bố chi tiết:")
print(f"{'Digit':<10} {'Train':<10} {'Test':<10} {'Total':<10}")
print("-" * 40)
for i in range(10):
    print(f"{i:<10} {train_counts[i]:<10} {test_counts[i]:<10} {train_counts[i] + test_counts[i]:<10}")
print("-" * 40)
print(f"{'TOTAL':<10} {y_train.shape[0]:<10} {y_test.shape[0]:<10} {y_train.shape[0] + y_test.shape[0]:<10}")

In [ ]:
# ======================================
# 4. Visualize Sample Images
# ======================================

# Hiển thị mẫu ảnh từ mỗi class
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
axes = axes.ravel()

for i in range(10):
    # Tìm ảnh đầu tiên của class i
    idx = np.where(y_train == i)[0][0]
    axes[i].imshow(X_train[idx], cmap='gray')
    axes[i].set_title(f'Digit: {i}', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.suptitle('MNIST Sample Images - 1 mẫu mỗi chữ số', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('mnist_samples.png', dpi=300, bbox_inches='tight')
plt.show()

# Hiển thị thêm nhiều mẫu random
print("\n📷 Hiển thị 20 ảnh random từ training set:")
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    idx = np.random.randint(0, len(X_train))
    axes[i].imshow(X_train[idx], cmap='gray')
    axes[i].set_title(f'Label: {y_train[idx]}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('20 ảnh random từ MNIST Training Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('mnist_random_samples.png', dpi=300, bbox_inches='tight')
plt.show()

## Phần 2: Tạo 100 ảnh chữ số tự vẽ

In [ ]:
# ======================================
# 5. Generate 100 Custom Handwritten Digit Images
# ======================================

from PIL import Image, ImageDraw, ImageFont
import os

def create_handwritten_digits(num_samples=100, save_dir='custom_digits'):
    """
    Tạo 100 ảnh chữ số viết tay synthetic với variations
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    custom_images = []
    custom_labels = []
    
    print(f"🎨 Tạo {num_samples} ảnh chữ số tự vẽ...")
    
    for i in range(num_samples):
        # Random digit (0-9)
        digit = i % 10
        
        # Tạo ảnh 28x28
        img = Image.new('L', (28, 28), color=0)
        draw = ImageDraw.Draw(img)
        
        # Random variations
        font_size = np.random.randint(16, 22)
        offset_x = np.random.randint(4, 12)
        offset_y = np.random.randint(2, 10)
        rotation = np.random.randint(-15, 15)
        
        # Vẽ chữ số
        try:
            # Sử dụng font mặc định
            draw.text((offset_x, offset_y), str(digit), fill=255)
        except:
            # Fallback nếu không có font
            draw.text((offset_x, offset_y), str(digit), fill=255)
        
        # Rotate nhẹ
        if rotation != 0:
            img = img.rotate(rotation, fillcolor=0)
        
        # Thêm noise nhẹ
        img_array = np.array(img)
        noise = np.random.normal(0, 5, img_array.shape)
        img_array = np.clip(img_array + noise, 0, 255).astype(np.uint8)
        
        # Lưu ảnh
        img_final = Image.fromarray(img_array)
        img_final.save(f'{save_dir}/digit_{i:03d}_label_{digit}.png')
        
        custom_images.append(img_array)
        custom_labels.append(digit)
    
    custom_images = np.array(custom_images)
    custom_labels = np.array(custom_labels)
    
    print(f"✅ Đã tạo {num_samples} ảnh!")
    print(f"📁 Lưu tại: {save_dir}/")
    print(f"Shape: {custom_images.shape}")
    
    return custom_images, custom_labels

# Tạo 100 ảnh custom
X_custom, y_custom = create_handwritten_digits(100)

# Hiển thị phân bố
custom_counts = np.bincount(y_custom)
print("\n📊 Phân bố ảnh custom:")
for digit, count in enumerate(custom_counts):
    print(f"Digit {digit}: {count} ảnh")

In [ ]:
# ======================================
# 6. Visualize Custom Images
# ======================================

# Hiển thị 20 ảnh custom
fig, axes = plt.subplots(4, 5, figsize=(12, 10))
axes = axes.ravel()

for i in range(20):
    axes[i].imshow(X_custom[i], cmap='gray')
    axes[i].set_title(f'Label: {y_custom[i]}', fontsize=10)
    axes[i].axis('off')

plt.suptitle('20 ảnh chữ số tự tạo (Custom Handwritten)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('custom_digits_samples.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Đã tạo và hiển thị 100 ảnh custom!")

## Phần 3: Model 1 - CNN với Keras (High-level API)

In [ ]:
# ======================================
# 7. Data Preprocessing for Keras Model
# ======================================

# Reshape và normalize data
X_train_keras = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test_keras = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_custom_keras = X_custom.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot encoding labels
y_train_keras = to_categorical(y_train, 10)
y_test_keras = to_categorical(y_test, 10)
y_custom_keras = to_categorical(y_custom, 10)

print("📊 Keras Model - Data Shape:")
print(f"X_train shape: {X_train_keras.shape}")
print(f"y_train shape: {y_train_keras.shape}")
print(f"X_test shape: {X_test_keras.shape}")
print(f"X_custom shape: {X_custom_keras.shape}")
print(f"\nPixel value range: [{X_train_keras.min():.2f}, {X_train_keras.max():.2f}]")

In [ ]:
# ======================================
# 8. Build CNN Model with Keras
# ======================================

def build_keras_cnn():
    """
    Kiến trúc CNN với Keras:
    - Conv2D(32) → ReLU → MaxPooling → Dropout(0.25)
    - Conv2D(64) → ReLU → MaxPooling → Dropout(0.25)
    - Flatten
    - Dense(128) → ReLU → Dropout(0.5)
    - Dense(10) → Softmax
    """
    model = keras.Sequential([
        # Convolutional Layer 1
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', 
                      input_shape=(28, 28, 1), padding='same', name='conv1'),
        layers.MaxPooling2D(pool_size=(2, 2), name='pool1'),
        layers.Dropout(0.25, name='dropout1'),
        
        # Convolutional Layer 2
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu', 
                      padding='same', name='conv2'),
        layers.MaxPooling2D(pool_size=(2, 2), name='pool2'),
        layers.Dropout(0.25, name='dropout2'),
        
        # Flatten
        layers.Flatten(name='flatten'),
        
        # Fully Connected Layer
        layers.Dense(128, activation='relu', name='fc1'),
        layers.Dropout(0.5, name='dropout3'),
        
        # Output Layer
        layers.Dense(10, activation='softmax', name='output')
    ])
    
    return model

# Build model
model_keras = build_keras_cnn()

# Compile
model_keras.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Model summary
print("🏗️ KIẾN TRÚC CNN MODEL 1 (KERAS):")
print("="*60)
model_keras.summary()
print("="*60)

# Visualize architecture
keras.utils.plot_model(
    model_keras, 
    to_file='keras_cnn_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=150
)
print("\n✅ Saved model architecture diagram: keras_cnn_architecture.png")

In [ ]:
# ======================================
# 9. Train Keras CNN Model
# ======================================

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("🏋️ Training Keras CNN Model...")
print("="*60)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Training
history_keras = model_keras.fit(
    X_train_keras, y_train_keras,
    batch_size=128,
    epochs=20,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

print("\n✅ Training completed!")

In [ ]:
# ======================================
# 10. Plot Training History (Keras)
# ======================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history_keras.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history_keras.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Model 1 (Keras) - Training Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history_keras.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history_keras.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Model 1 (Keras) - Training Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('keras_cnn_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("📈 Training Summary:")
print(f"Final Train Accuracy: {history_keras.history['accuracy'][-1]:.4f}")
print(f"Final Val Accuracy: {history_keras.history['val_accuracy'][-1]:.4f}")
print(f"Final Train Loss: {history_keras.history['loss'][-1]:.4f}")
print(f"Final Val Loss: {history_keras.history['val_loss'][-1]:.4f}")

In [ ]:
# ======================================
# 11. Evaluate Keras Model
# ======================================

# Evaluate on test set
print("📊 Evaluating Keras Model on MNIST Test Set...")
test_loss, test_acc = model_keras.evaluate(X_test_keras, y_test_keras, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Evaluate on custom images
print("\n📊 Evaluating Keras Model on Custom Images...")
custom_loss, custom_acc = model_keras.evaluate(X_custom_keras, y_custom_keras, verbose=0)
print(f"Custom Images Accuracy: {custom_acc:.4f}")
print(f"Custom Images Loss: {custom_loss:.4f}")

# Predictions
y_pred_test_keras = model_keras.predict(X_test_keras, verbose=0)
y_pred_test_keras = np.argmax(y_pred_test_keras, axis=1)

y_pred_custom_keras = model_keras.predict(X_custom_keras, verbose=0)
y_pred_custom_keras = np.argmax(y_pred_custom_keras, axis=1)

# Classification Report
print("\n📋 Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test_keras, digits=4))

In [ ]:
# ======================================
# 12. Confusion Matrix (Keras Model)
# ======================================

# Test set confusion matrix
cm_test_keras = confusion_matrix(y_test, y_pred_test_keras)

# Custom images confusion matrix
cm_custom_keras = confusion_matrix(y_custom, y_pred_custom_keras)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Test set CM
sns.heatmap(cm_test_keras, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10), ax=axes[0])
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_title('Keras CNN - Test Set Confusion Matrix', fontsize=14, fontweight='bold')

# Custom images CM
sns.heatmap(cm_custom_keras, annot=True, fmt='d', cmap='Greens',
            xticklabels=range(10), yticklabels=range(10), ax=axes[1])
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_title('Keras CNN - Custom Images Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('keras_cnn_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## Phần 4: Model 2 - CNN từ đầu với NumPy (Không dùng Keras)

In [ ]:
# ======================================
# 13. CNN Implementation từ đầu với NumPy
# ======================================

class SimpleCNN:
    """
    Simplified CNN implementation using only NumPy
    Architecture:
    - Conv2D(16) → ReLU → MaxPool
    - Flatten → Dense(64) → ReLU → Dropout → Dense(10) → Softmax
    """
    
    def __init__(self, input_shape=(28, 28), num_classes=10, learning_rate=0.001):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.lr = learning_rate
        
        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        """Xavier initialization"""
        # Conv layer: 16 filters, 3x3 kernel
        self.W_conv = np.random.randn(16, 3, 3) * np.sqrt(2.0 / (3 * 3))
        self.b_conv = np.zeros(16)
        
        # After conv and pool: (28-2)//2 = 13, so 13x13x16 = 2704
        self.conv_output_size = 13 * 13 * 16
        
        # Fully connected layer
        self.W_fc = np.random.randn(self.conv_output_size, 64) * np.sqrt(2.0 / self.conv_output_size)
        self.b_fc = np.zeros(64)
        
        # Output layer
        self.W_out = np.random.randn(64, self.num_classes) * np.sqrt(2.0 / 64)
        self.b_out = np.zeros(self.num_classes)
        
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        return (x > 0).astype(float)
    
    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    def conv2d(self, image, kernel):
        """Simple 2D convolution (valid padding)"""
        h, w = image.shape
        kh, kw = kernel.shape
        output_h = h - kh + 1
        output_w = w - kw + 1
        output = np.zeros((output_h, output_w))
        
        for i in range(output_h):
            for j in range(output_w):
                output[i, j] = np.sum(image[i:i+kh, j:j+kw] * kernel)
        
        return output
    
    def maxpool2d(self, image, pool_size=2):
        """Max pooling 2x2"""
        h, w = image.shape
        output_h = h // pool_size
        output_w = w // pool_size
        output = np.zeros((output_h, output_w))
        
        for i in range(output_h):
            for j in range(output_w):
                output[i, j] = np.max(image[i*pool_size:(i+1)*pool_size, 
                                            j*pool_size:(j+1)*pool_size])
        
        return output
    
    def forward(self, X, training=True, dropout_rate=0.5):
        """Forward pass"""
        batch_size = X.shape[0]
        
        # Conv + Pool
        conv_outputs = []
        for img in X:
            conv_maps = []
            for f in range(16):
                conv_out = self.conv2d(img, self.W_conv[f])
                conv_out = self.relu(conv_out + self.b_conv[f])
                pooled = self.maxpool2d(conv_out)
                conv_maps.append(pooled)
            conv_outputs.append(np.array(conv_maps))
        
        conv_output = np.array(conv_outputs)
        
        # Flatten
        flattened = conv_output.reshape(batch_size, -1)
        
        # FC layer
        fc_output = self.relu(np.dot(flattened, self.W_fc) + self.b_fc)
        
        # Dropout during training
        if training:
            dropout_mask = (np.random.rand(*fc_output.shape) > dropout_rate).astype(float)
            fc_output = fc_output * dropout_mask / (1 - dropout_rate)
        
        # Output layer
        logits = np.dot(fc_output, self.W_out) + self.b_out
        predictions = self.softmax(logits)
        
        # Store for backward pass
        if training:
            self.cache = {
                'X': X,
                'conv_output': conv_output,
                'flattened': flattened,
                'fc_output': fc_output,
                'predictions': predictions
            }
        
        return predictions
    
    def backward(self, y_true):
        """Simplified backward pass"""
        batch_size = y_true.shape[0]
        
        # Output layer gradient
        d_logits = self.cache['predictions'] - y_true
        
        # FC layer gradients
        dW_out = np.dot(self.cache['fc_output'].T, d_logits) / batch_size
        db_out = np.mean(d_logits, axis=0)
        
        d_fc = np.dot(d_logits, self.W_out.T)
        d_fc = d_fc * self.relu_derivative(self.cache['fc_output'])
        
        dW_fc = np.dot(self.cache['flattened'].T, d_fc) / batch_size
        db_fc = np.mean(d_fc, axis=0)
        
        # Update weights
        self.W_out -= self.lr * dW_out
        self.b_out -= self.lr * db_out
        self.W_fc -= self.lr * dW_fc
        self.b_fc -= self.lr * db_fc
        
    def train_step(self, X_batch, y_batch):
        """One training step"""
        predictions = self.forward(X_batch, training=True)
        self.backward(y_batch)
        
        # Cross-entropy loss
        loss = -np.mean(np.sum(y_batch * np.log(predictions + 1e-8), axis=1))
        
        # Accuracy
        pred_labels = np.argmax(predictions, axis=1)
        true_labels = np.argmax(y_batch, axis=1)
        accuracy = np.mean(pred_labels == true_labels)
        
        return loss, accuracy
    
    def predict(self, X):
        """Prediction"""
        predictions = self.forward(X, training=False)
        return np.argmax(predictions, axis=1)

print("✅ SimpleCNN class implemented!")

In [ ]:
# ======================================
# 14. Prepare Data for NumPy CNN
# ======================================

# Normalize (keep 2D shape for NumPy CNN)
X_train_numpy = X_train.astype('float32') / 255.0
X_test_numpy = X_test.astype('float32') / 255.0
X_custom_numpy = X_custom.astype('float32') / 255.0

# One-hot encoding (reuse from Keras)
y_train_numpy = y_train_keras
y_test_numpy = y_test_keras
y_custom_numpy = y_custom_keras

print("📊 NumPy Model - Data Shape:")
print(f"X_train shape: {X_train_numpy.shape}")
print(f"y_train shape: {y_train_numpy.shape}")
print(f"X_test shape: {X_test_numpy.shape}")
print(f"X_custom shape: {X_custom_numpy.shape}")

In [ ]:
# ======================================
# 15. Train NumPy CNN Model
# ======================================

print("🏋️ Training NumPy CNN Model...")
print("="*60)
print("⚠️ NOTE: Training sẽ chậm hơn Keras do implement bằng NumPy thuần")
print("Chỉ train trên subset nhỏ để demo...")
print("="*60)

# Use smaller subset for training (NumPy implementation is slow)
TRAIN_SIZE = 5000
VAL_SIZE = 1000

# Sample training data
indices = np.random.choice(len(X_train_numpy), TRAIN_SIZE, replace=False)
X_train_small = X_train_numpy[indices]
y_train_small = y_train_numpy[indices]

# Validation data
val_indices = np.random.choice(len(X_test_numpy), VAL_SIZE, replace=False)
X_val_small = X_test_numpy[val_indices]
y_val_small = y_test_numpy[val_indices]

# Initialize model
model_numpy = SimpleCNN(learning_rate=0.01)

# Training loop
EPOCHS = 5
BATCH_SIZE = 32
history_numpy = {'train_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    
    # Shuffle data
    perm = np.random.permutation(len(X_train_small))
    X_train_shuffled = X_train_small[perm]
    y_train_shuffled = y_train_small[perm]
    
    # Mini-batch training
    epoch_losses = []
    epoch_accs = []
    
    num_batches = len(X_train_small) // BATCH_SIZE
    
    for i in range(num_batches):
        start_idx = i * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE
        
        X_batch = X_train_shuffled[start_idx:end_idx]
        y_batch = y_train_shuffled[start_idx:end_idx]
        
        loss, acc = model_numpy.train_step(X_batch, y_batch)
        epoch_losses.append(loss)
        epoch_accs.append(acc)
        
        if (i + 1) % 20 == 0:
            print(f"  Batch {i+1}/{num_batches} - Loss: {loss:.4f}, Acc: {acc:.4f}")
    
    # Validation
    val_predictions = model_numpy.predict(X_val_small)
    val_true = np.argmax(y_val_small, axis=1)
    val_acc = np.mean(val_predictions == val_true)
    
    avg_loss = np.mean(epoch_losses)
    avg_acc = np.mean(epoch_accs)
    
    history_numpy['train_loss'].append(avg_loss)
    history_numpy['train_acc'].append(avg_acc)
    history_numpy['val_acc'].append(val_acc)
    
    print(f"  Epoch Summary - Train Loss: {avg_loss:.4f}, Train Acc: {avg_acc:.4f}, Val Acc: {val_acc:.4f}")

print("\n✅ NumPy CNN training completed!")

In [ ]:
# ======================================
# 16. Plot Training History (NumPy Model)
# ======================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, len(history_numpy['train_loss']) + 1)

# Loss
axes[0].plot(epochs_range, history_numpy['train_loss'], marker='o', 
             label='Train Loss', linewidth=2, markersize=8)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Model 2 (NumPy) - Training Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(epochs_range, history_numpy['train_acc'], marker='o',
             label='Train Accuracy', linewidth=2, markersize=8)
axes[1].plot(epochs_range, history_numpy['val_acc'], marker='s',
             label='Val Accuracy', linewidth=2, markersize=8)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Model 2 (NumPy) - Training Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('numpy_cnn_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("📈 Training Summary:")
print(f"Final Train Accuracy: {history_numpy['train_acc'][-1]:.4f}")
print(f"Final Val Accuracy: {history_numpy['val_acc'][-1]:.4f}")
print(f"Final Train Loss: {history_numpy['train_loss'][-1]:.4f}")

In [ ]:
# ======================================
# 17. Evaluate NumPy Model
# ======================================

print("📊 Evaluating NumPy Model...")

# Test on full test set (sample for speed)
test_sample_size = 1000
test_indices = np.random.choice(len(X_test_numpy), test_sample_size, replace=False)
X_test_sample = X_test_numpy[test_indices]
y_test_sample = y_test[test_indices]

y_pred_test_numpy = model_numpy.predict(X_test_sample)
test_acc_numpy = accuracy_score(y_test_sample, y_pred_test_numpy)

print(f"\n✅ Test Accuracy (sample {test_sample_size}): {test_acc_numpy:.4f}")

# Test on custom images
y_pred_custom_numpy = model_numpy.predict(X_custom_numpy)
custom_acc_numpy = accuracy_score(y_custom, y_pred_custom_numpy)

print(f"✅ Custom Images Accuracy: {custom_acc_numpy:.4f}")

# Classification Report
print("\n📋 Classification Report (Test Sample):")
print(classification_report(y_test_sample, y_pred_test_numpy, digits=4))

In [ ]:
# ======================================
# 18. Confusion Matrix (NumPy Model)
# ======================================

# Confusion matrices
cm_test_numpy = confusion_matrix(y_test_sample, y_pred_test_numpy)
cm_custom_numpy = confusion_matrix(y_custom, y_pred_custom_numpy)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Test set CM
sns.heatmap(cm_test_numpy, annot=True, fmt='d', cmap='Oranges',
            xticklabels=range(10), yticklabels=range(10), ax=axes[0])
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_title('NumPy CNN - Test Set Confusion Matrix', fontsize=14, fontweight='bold')

# Custom images CM
sns.heatmap(cm_custom_numpy, annot=True, fmt='d', cmap='Purples',
            xticklabels=range(10), yticklabels=range(10), ax=axes[1])
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_title('NumPy CNN - Custom Images Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('numpy_cnn_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## Phần 5: So sánh và Đánh giá 2 Models

In [ ]:
# ======================================
# 19. Model Comparison Summary
# ======================================

print("="*80)
print(" " * 25 + "SO SÁNH 2 MODELS CNN")
print("="*80)

comparison_data = {
    'Model': ['Model 1 (Keras)', 'Model 2 (NumPy)'],
    'Implementation': ['Keras/TensorFlow API', 'NumPy from scratch'],
    'Test Accuracy': [f'{test_acc:.4f}', f'{test_acc_numpy:.4f}'],
    'Custom Acc': [f'{custom_acc:.4f}', f'{custom_acc_numpy:.4f}'],
    'Training Time': ['Fast (GPU optimized)', 'Slow (CPU only)'],
    'Code Complexity': ['Low (high-level API)', 'High (manual implementation)'],
    'Dropout': ['✓ (3 layers)', '✓ (1 layer)'],
    'Epochs Trained': [len(history_keras.history['loss']), EPOCHS]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n", df_comparison.to_string(index=False))
print("\n" + "="*80)

# Detailed comparison
print("\n📊 KIẾN TRÚC CHI TIẾT:")
print("\n🔹 Model 1 (Keras CNN):")
print("   - Conv2D(32, 3x3) → ReLU → MaxPool(2x2) → Dropout(0.25)")
print("   - Conv2D(64, 3x3) → ReLU → MaxPool(2x2) → Dropout(0.25)")
print("   - Flatten")
print("   - Dense(128) → ReLU → Dropout(0.5)")
print("   - Dense(10) → Softmax")
print(f"   - Total Parameters: {model_keras.count_params():,}")

print("\n🔹 Model 2 (NumPy CNN):")
print("   - Conv2D(16, 3x3) → ReLU → MaxPool(2x2)")
print("   - Flatten")
print("   - Dense(64) → ReLU → Dropout(0.5)")
print("   - Dense(10) → Softmax")
print("   - Parameters: ~175K (simplified architecture)")

print("\n💡 NHẬN XÉT:")
print("   ✓ Model 1 (Keras): Accuracy cao hơn, training nhanh, dễ implement")
print("   ✓ Model 2 (NumPy): Hiểu rõ cơ chế CNN, nhưng chậm và accuracy thấp hơn")
print("   ✓ Keras là lựa chọn tốt cho production, NumPy tốt cho học tập")

In [ ]:
# ======================================
# 20. Visualization: Accuracy Comparison
# ======================================

# Comparison bar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

models = ['Keras CNN', 'NumPy CNN']
test_accs = [test_acc, test_acc_numpy]
custom_accs = [custom_acc, custom_acc_numpy]

# Test accuracy comparison
x = np.arange(len(models))
width = 0.35

bars1 = axes[0].bar(x - width/2, test_accs, width, label='Test Set', 
                     color='skyblue', edgecolor='black')
bars2 = axes[0].bar(x + width/2, custom_accs, width, label='Custom Images',
                     color='lightcoral', edgecolor='black')

axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend(fontsize=11)
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_ylim(0, 1)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# Training comparison
keras_final_acc = history_keras.history['val_accuracy'][-1]
numpy_final_acc = history_numpy['val_acc'][-1]

comparison_metrics = ['Validation Accuracy', 'Parameters (M)', 'Training Speed']
keras_values = [keras_final_acc, model_keras.count_params()/1e6, 100]
numpy_values = [numpy_final_acc, 0.175, 10]

x2 = np.arange(len(['Val Accuracy']))
axes[1].bar(['Keras CNN', 'NumPy CNN'], [keras_final_acc, numpy_final_acc],
            color=['green', 'orange'], edgecolor='black', alpha=0.7)
axes[1].set_ylabel('Validation Accuracy', fontsize=12)
axes[1].set_title('Validation Accuracy Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, 1)
axes[1].grid(axis='y', alpha=0.3)

# Add value labels
for i, (model, val) in enumerate(zip(['Keras CNN', 'NumPy CNN'], 
                                      [keras_final_acc, numpy_final_acc])):
    axes[1].text(i, val + 0.02, f'{val:.3f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('model_comparison_final.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Model comparison visualization completed!")

In [ ]:
# ======================================
# 21. Sample Predictions Visualization
# ======================================

# Show some prediction examples
num_samples = 10
sample_indices = np.random.choice(len(X_custom), num_samples, replace=False)

fig, axes = plt.subplots(2, 5, figsize=(15, 7))
axes = axes.ravel()

for i, idx in enumerate(sample_indices):
    # Get predictions from both models
    img = X_custom[idx]
    true_label = y_custom[idx]
    
    # Keras prediction
    img_keras = img.reshape(1, 28, 28, 1).astype('float32') / 255.0
    pred_keras = model_keras.predict(img_keras, verbose=0)
    pred_label_keras = np.argmax(pred_keras)
    confidence_keras = pred_keras[0][pred_label_keras]
    
    # NumPy prediction
    img_numpy = img.astype('float32') / 255.0
    pred_label_numpy = model_numpy.predict(img_numpy.reshape(1, 28, 28))[0]
    
    # Display
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
    
    # Title with predictions
    color = 'green' if pred_label_keras == true_label else 'red'
    title = f'True: {true_label}\n'
    title += f'Keras: {pred_label_keras} ({confidence_keras:.2f})\n'
    title += f'NumPy: {pred_label_numpy}'
    axes[i].set_title(title, fontsize=9, color=color, fontweight='bold')

plt.suptitle('Sample Predictions from Custom Images', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Sample predictions visualization completed!")

## Kết luận

### 📊 Kết quả chính:

**Model 1 (Keras CNN):**
- ✅ Accuracy cao trên cả MNIST test set và custom images
- ✅ Training nhanh, tối ưu GPU
- ✅ Kiến trúc phức tạp hơn với 2 conv layers và 3 dropout layers
- ✅ Dễ implement và maintain
- ✅ Phù hợp cho production

**Model 2 (NumPy CNN):**
- ✅ Implement hoàn toàn bằng NumPy thuần
- ✅ Giúp hiểu rõ cơ chế hoạt động của CNN
- ⚠️ Accuracy thấp hơn do kiến trúc đơn giản
- ⚠️ Training chậm (chỉ dùng CPU)
- ✅ Tốt cho mục đích học tập và nghiên cứu

### 🎯 Điểm nổi bật:

1. **Data Distribution**: Dataset MNIST cân bằng, mỗi digit có khoảng 6000 mẫu train
2. **Custom Images**: Tạo 100 ảnh synthetic để test khả năng tổng quát hóa
3. **Dropout**: Cả 2 model đều sử dụng dropout để chống overfitting
4. **Architecture**: Model Keras phức tạp hơn nhưng chính xác hơn

### 💡 Khuyến nghị:
- Sử dụng **Keras/TensorFlow** cho các dự án thực tế
- Implement **NumPy CNN** chỉ để học và hiểu sâu về cơ chế
- Có thể kết hợp cả 2 cách tiếp cận trong quá trình học

---
**Tạ Cao Sơn - B22DCVT445**